# 🚗 YOLOv8 Pothole & Crack Detection — Training on Colab

This notebook trains YOLOv8 for pothole and crack detection using a **free T4 GPU** on Google Colab.

## Steps:
1. Connect to GPU runtime
2. Clone the GitHub repo
3. Install dependencies
4. Download datasets from Roboflow
5. Train (2-stage transfer learning)
6. Download trained model

---

> ⚠️ **First**: Go to **Runtime → Change runtime type → T4 GPU** before running any cells.

## 1. Verify GPU

In [ ]:
!nvidia-smi
import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

## 2. Clone Repository & Install Dependencies

In [ ]:
import os

# Clone the repo
REPO_URL = "https://github.com/Shubhamf0073/Margdrashti_models.git"
REPO_DIR = "/content/Margdrashti_models"

if os.path.exists(REPO_DIR):
    print(f"Repo already cloned at {REPO_DIR}, pulling latest...")
    !cd {REPO_DIR} && git pull
else:
    !git clone {REPO_URL} {REPO_DIR}

os.chdir(REPO_DIR)
print(f"\nWorking directory: {os.getcwd()}")
!ls -la

In [ ]:
# Install required packages
!pip install -q ultralytics roboflow pyyaml

# Verify installation
import ultralytics
print(f"Ultralytics: {ultralytics.__version__}")

## 3. Download Dataset from Roboflow

In [ ]:
# ============================================================
# 🔑 SET YOUR ROBOFLOW API KEY HERE
# ============================================================
ROBOFLOW_KEY = "lcZrl8Z8hFNZYgrdN33C"  # <-- Replace if needed

!python scripts/yolov8_detection/download_datasets.py \
    --roboflow_key {ROBOFLOW_KEY} \
    --output_dir data/pothole_crack_detection \
    --skip_kaggle

In [ ]:
# Verify dataset
!cat data/pothole_crack_detection/data.yaml
print()
!echo "Train images:" && ls data/pothole_crack_detection/images/train/ | wc -l
!echo "Val images:"   && ls data/pothole_crack_detection/images/val/ | wc -l
!echo "Test images:"  && ls data/pothole_crack_detection/images/test/ | wc -l

## 4. Fix data.yaml Path for Colab

The data.yaml has an absolute path from local machine. We need to update it for Colab.

In [ ]:
import yaml

yaml_path = "data/pothole_crack_detection/data.yaml"

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Update to Colab absolute path
data['path'] = f"{REPO_DIR}/data/pothole_crack_detection"

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print("Updated data.yaml:")
!cat {yaml_path}

## 5. Train YOLOv8 (Two-Stage Transfer Learning)

- **Stage 1** (15 epochs): Backbone frozen, trains detection head only
- **Stage 2** (30 epochs): Full fine-tuning with lower learning rate

On a T4 GPU this should take **~20-40 minutes** total.

In [ ]:
!python scripts/yolov8_detection/train_yolov8.py \
    --data data/pothole_crack_detection/data.yaml \
    --model yolov8n.pt \
    --epochs_stage1 15 \
    --epochs_stage2 30 \
    --batch 32 \
    --device 0 \
    --cache

## 6. View Training Results

In [ ]:
from IPython.display import Image, display
import glob

# Show training curves
for stage in ['yolov8n_stage1', 'yolov8n_stage2']:
    results_img = f"scripts/runs/{stage}/results.png"
    if os.path.exists(results_img):
        print(f"\n{'='*50}")
        print(f"{stage} Results")
        print(f"{'='*50}")
        display(Image(filename=results_img, width=800))

# Show confusion matrix if available
for stage in ['yolov8n_stage1', 'yolov8n_stage2']:
    cm_img = f"scripts/runs/{stage}/confusion_matrix.png"
    if os.path.exists(cm_img):
        print(f"\n{stage} Confusion Matrix")
        display(Image(filename=cm_img, width=600))

## 7. Download Trained Model

Download the best model weights to your local machine.

In [ ]:
from google.colab import files
import shutil

# Path to the best model from Stage 2
best_model = "scripts/runs/yolov8n_stage2/weights/best.pt"

if os.path.exists(best_model):
    print(f"Best model: {best_model}")
    print(f"Size: {os.path.getsize(best_model) / 1024 / 1024:.1f} MB")
    
    # Also zip the entire runs directory for full results
    shutil.make_archive("/content/training_results", 'zip', "scripts/runs")
    
    print("\n📥 Downloading best model weight...")
    files.download(best_model)
    
    print("\n📥 Downloading full results archive...")
    files.download("/content/training_results.zip")
else:
    # Fallback to Stage 1 model
    fallback = "scripts/runs/yolov8n_stage1/weights/best.pt"
    if os.path.exists(fallback):
        print(f"Stage 2 not found, downloading Stage 1 model: {fallback}")
        files.download(fallback)
    else:
        print("❌ No trained model found. Check training output above for errors.")

## 8. (Optional) Test Inference on Colab

Quick test on a sample image from the test set.

In [ ]:
from ultralytics import YOLO
from IPython.display import Image, display

# Load trained model
model = YOLO("scripts/runs/yolov8n_stage2/weights/best.pt")

# Get a test image
test_images = glob.glob("data/pothole_crack_detection/images/test/*")
if test_images:
    results = model.predict(
        source=test_images[:5],
        conf=0.5,
        save=True,
        project="test_predictions",
        name="samples",
        exist_ok=True
    )
    
    # Display predictions
    for img_path in glob.glob("test_predictions/samples/*"):
        display(Image(filename=img_path, width=600))
else:
    print("No test images found")

---
## ✅ Next Steps (on your local machine)

After downloading `best.pt`, place it at `scripts/runs/yolov8n_stage2/weights/best.pt` in your local repo and run:

```bash
# On a video file
python deploy_inference_yolov8.py \
    --model scripts/runs/yolov8n_stage2/weights/best.pt \
    --roi roi_highway_shorter.json \
    --source path/to/your/video.mp4 \
    --confidence 0.5 \
    --save_output results/output.mp4

# On webcam
python deploy_inference_yolov8.py \
    --model scripts/runs/yolov8n_stage2/weights/best.pt \
    --roi roi_highway_shorter.json \
    --source 0
```